In [62]:
# Run this block to install dependencies [Remember to make the statement true]
if 0 == 1:
    !pip3 install pandas
    !pip3 install tqdm
    !pip3 install scikit-learn
    !pip3 install gensim
    !pip3 install spacy
    !python3 -m spacy download en
    !pip3 install numpy

In [63]:
import pandas as pd
import spacy
import pickle
from tqdm import tqdm
from gensim.corpora import Dictionary
from sklearn.utils import shuffle
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

nlp = spacy.load('en')
DATA_LIMIT = 1000

df = pd.read_csv('./imdb_master.csv', encoding='latin1')
df_neg = df[df['label'] == 'neg']
df_pos = df[df['label'] == 'pos']
df = pd.concat((df_pos[:DATA_LIMIT], df_neg[:DATA_LIMIT]))

def process_text(input_string, return_string=False, stem=False):
    text = nlp(u'' + input_string)
    if stem == True:
        text = [tok.lemma_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    else:
        text = [tok.lower_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    if return_string == True:
        return " ".join(text)
    return text



    Only loading the 'en' tokenizer.



In [64]:
# Make this statement true to run from scratch [It takes time to process the text]
if 1 == 0:
    wordlist = []
    for i in tqdm(range(df.shape[0])):
        wordlist.append(process_text(df['review'].iloc[i]))
        
    with open('vocabulary.txt', 'wb') as vocabulary:
        pickle.dump(wordlist, vocabulary)
    vocabulary.close()

In [65]:
# Load vocabulary
wordlist = []
with open('vocabulary.txt', 'rb') as vocabulary:
    wordlist = pickle.load(vocabulary)

In [66]:
# Keeping track of frequency of a single token
frequency = defaultdict(int)
for text in wordlist:
    for token in text:
        frequency[token] += 1
        
# Apply Threshold to limit the vocabulary size, discarding the tokens which appeard number of times below the threshold limit 
FREQ_THRESHOLD = 5

thresholded_wordlist =  [[token for token in text if frequency[token] > FREQ_THRESHOLD]
          for text in wordlist]

# Create Dictionary based on the word list
dictionary = Dictionary(thresholded_wordlist)

# Number of tokens
print("Number of Tokens - {}".format(len(dictionary)))

Number of Tokens - 5464


In [67]:
print(dictionary)

Dictionary(5464 unique tokens: ['able', 'admit', 'ben', 'character', 'comedy']...)


![image](https://i.imgur.com/f1uzTDZ.png)

# TODO

* From the screenshot you can see the implementation of word-cooccurance matrix, based on the tokens from the dictionary, build a word-cooccurance matrix yourself which is $X$. Documentation of gensim [https://radimrehurek.com/gensim/corpora/dictionary.html]
* Apply SVD on $X$
* Reduce Dimension 

![dimen_reduc](https://i.imgur.com/lezB870.png)

* Here Richard is taking only top two dimensions of the vector $U$, recommended size is *50* for now.

![dimen_reduc_u](https://i.imgur.com/TA2Bmsq.png)

* Now we can get a fixed size vector for each word. 

* Try to plot something similar based on the given dataset. In class we will try to implement a logistic regression classifier that can classify positive and negative reviews

In [68]:
len_d=len(dictionary)
X=np.zeros((len_d,len_d))
la=np.linalg
for sen in thresholded_wordlist:
    
    idx=dictionary.doc2idx(sen)
    for i in idx:
        for j in idx:
            X[i,j]=X[i,j]+1
U,s,Vh = la.svd(X, full_matrices=False)


In [69]:
print(U.shape)


(5464, 5464)


In [70]:
print(s.shape)

(5464,)


In [71]:
print(Vh.shape)


(5464, 5464)


In [74]:
for i in range((len_d)):
    plt.text(U_new[i,0], U_new[i,1], dictionary[i])
    

    #how to plot this 